<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

# A Baseline Model for the Radiant Earth Spot the Crop Challenge

This notebook walks you through the steps to load the data and build a baseline model using Random Forests for `Radiant Earth Spot the Crop Challenge`.

## Radiant MLHub API


The Radiant MLHub API gives access to open Earth imagery training data for machine learning applications. You can learn more about the repository at the [Radiant MLHub site](https://mlhub.earth) and about the organization behind it at the [Radiant Earth Foundation site](https://radiant.earth).

Full documentation for the API is available at [https://mlhub.earth/docs](https://mlhub.earth/docs).

Each item in our collection is explained in json format compliant with [STAC](https://stacspec.org/) [label extension](https://github.com/stac-extensions/label) definition.

## Dependencies

This notebook utilizes the [`radiant-mlhub` Python client](https://pypi.org/project/radiant-mlhub/) for interacting with the API. This notebook also utilizes the [`pandas` library](https://pandas.pydata.org/). If you are running this notebooks using Binder, then these dependencies have already been installed. If you are running this notebook locally, you will need to install these yourself.

See the official [`radiant-mlhub` docs](https://radiant-mlhub.readthedocs.io/) for more documentation of the full functionality of that library.

In [9]:
# Required libraries
import os
import tarfile
import json
import pandas as pd
from pathlib import Path
from radiant_mlhub.client.datasets import download_collection_archive

import datetime
import rasterio
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit

os.environ['MLHUB_API_KEY'] = 'N/A'

In [3]:
DOWNLOAD_S1 = True # If you set this to true then the Sentinel-1 data will be downloaded

# Select which Sentinel-2 imagery bands you'd like to download here. 
DOWNLOAD_BANDS = {
    'B01': False,
    'B02': False,
    'B03': False,
    'B04': False,
    'B05': False,
    'B06': False,
    'B07': False,
    'B08': False,
    'B8A': False,
    'B09': False,
    'B11': False,
    'B12': False,
    'CLM': False
}

## Downloading and Loading the Data

In this part, we will download the data from Radiant MLHub and load the properties of each item in the dataset into a DataFrame


In [11]:
FOLDER_BASE = 'ref_south_africa_crops_competition_v1'

def download_archive(archive_name):
    if os.path.exists(archive_name.replace('.tar.gz', '')):
        return
    
    print(f'Downloading {archive_name} ...')
    download_url = f'https://radiant-mlhub.s3.us-west-2.amazonaws.com/archives/{archive_name}'
    download_collection_archive(download_url, '.')
    print(f'Extracting {archive_name} ...')
    with tarfile.open(archive_name) as tfile:
        tfile.extractall()
    os.remove(archive_name)

for split in ['train', 'test']:
    # Download the labels
    labels_archive = f'{FOLDER_BASE}_{split}_labels.tar.gz'
    download_archive(labels_archive)
    
    # Download Sentinel-1 data
    if DOWNLOAD_S1:
        s1_archive = f'{FOLDER_BASE}_{split}_source_s1.tar.gz'
        download_archive(s1_archive)
        

    for band, download in DOWNLOAD_BANDS.items():
        if not download:
            continue
        s2_archive = f'{FOLDER_BASE}_{split}_source_s2_{band}.tar.gz'
        download_archive(s2_archive)
        
def resolve_path(base, path):
    return Path(os.path.join(base, path)).resolve()
        
def load_df(collection_id):
    split = collection_id.split('_')[-2]
    collection = json.load(open(f'{collection_id}/collection.json', 'r'))
    rows = []
    item_links = []
    for link in collection['links']:
        if link['rel'] != 'item':
            continue
        item_links.append(link['href'])
        
    for item_link in item_links:
        item_path = f'{collection_id}/{item_link}'
        current_path = os.path.dirname(item_path)
        item = json.load(open(item_path, 'r'))
        tile_id = item['id'].split('_')[-1]
        for asset_key, asset in item['assets'].items():
            rows.append([
                tile_id,
                None,
                None,
                asset_key,
                str(resolve_path(current_path, asset['href']))
            ])
            
        for link in item['links']:
            if link['rel'] != 'source':
                continue
            source_item_id = link['href'].split('/')[-2]
            
            if source_item_id.find('_s1_') > 0 and not DOWNLOAD_S1:
                continue
            elif source_item_id.find('_s1_') > 0:
                for band in ['VV', 'VH']:
                    asset_path = Path(f'{FOLDER_BASE}_{split}_source_s1/{source_item_id}/{band}.tif').resolve()
                    date = '-'.join(source_item_id.split('_')[10:13])
                    
                    rows.append([
                        tile_id,
                        f'{date}T00:00:00Z',
                        's1',
                        band,
                        asset_path
                    ])
                
            if source_item_id.find('_s2_') > 0:
                for band, download in DOWNLOAD_BANDS.items():
                    if not download:
                        continue
                    
                    asset_path = Path(f'{FOLDER_BASE}_{split}_source_s2_{band}/{source_item_id}_{band}.tif').resolve()
                    date = '-'.join(source_item_id.split('_')[10:13])
                    rows.append([
                        tile_id,
                        f'{date}T00:00:00Z',
                        's2',
                        band,
                        asset_path
                    ])
            
    return pd.DataFrame(rows, columns=['tile_id', 'datetime', 'satellite_platform', 'asset', 'file_path'])

competition_train_df = load_df(f'{FOLDER_BASE}_train_labels')
competition_test_df = load_df(f'{FOLDER_BASE}_test_labels')

RetryError: HTTPSConnectionPool(host='api.radiant.earth', port=443): Max retries exceeded with url: /mlhub/v1/archive/https:/radiant-mlhub.s3.us-west-2.amazonaws.com/archives/ref_south_africa_crops_competition_v1_train_labels.tar.gz?key=N%2FA (Caused by ResponseError('too many 404 error responses'))

In [4]:
competition_train_df

,tile_id,datetime,satellite_platform,asset,file_path
0,2587,None,None,documentation,/Users/hamed/Dropbox (Radiant)/Radiant/mlhub/m...
1,2587,None,None,field_ids,/Users/hamed/Dropbox (Radiant)/Radiant/mlhub/m...
2,2587,None,None,field_info_train,/Users/hamed/Dropbox (Radiant)/Radiant/mlhub/m...
3,2587,None,None,labels,/Users/hamed/Dropbox (Radiant)/Radiant/mlhub/m...
4,2587,None,None,raster_values,/Users/hamed/Dropbox (Radiant)/Radiant/mlhub/m...
...,...,...,...,...,...
161669,2198,2017-11-03T00:00:00Z,s1,VH,/Users/hamed/Dropbox (Radiant)/Radiant/mlhub/m...
161670,2198,2017-11-15T00:00:00Z,s1,VV,/Users/hamed/Dropbox (Radiant)/Radiant/mlhub/m...
161671,2198,2017-11-15T00:00:00Z,s1,VH,/Users/hamed/Dropbox (Radiant)/Radiant/mlhub/m...
161672,2198,2017-11-27T00:00:00Z,s1,VV,/Users/hamed/Dropbox (Radiant)/Radiant/mlhub/m...


In [5]:
# This DataFrame lists all types of assets including documentation of the data. 
# In the following, we will use the Sentinel-1 bands (VV and VH) as well as labels. 
competition_train_df['asset'].unique()

array(['documentation', 'field_ids', 'field_info_train', 'labels',
       'raster_values', 'VH', 'VV'], dtype=object)

In [6]:
tile_ids_train = competition_train_df['tile_id'].unique()

In [7]:
# For simplicty of this baseline model, we will use only 5 observations throughout the growing season
# You can choose to use all of them, select a few of them at specifc intervals, or 
# load as many as you want and interpolate between them to have a regular temporal frequency.
n_obs = 5

In [ ]:
X = np.empty((0, 2 * (n_obs-1)))
y = np.empty((0, 1))
field_ids = np.empty((0, 1))

for tile_id in tile_ids_train:
    tile_df = competition_train_df[competition_train_df['tile_id']==tile_id]

    label_src = rasterio.open(tile_df[tile_df['asset']=='labels']['file_path'].values[0])
    label_array = label_src.read(1)
    y = np.append(y, label_array.flatten())

    field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
    field_id_array = field_id_src.read(1)
    field_ids = np.append(field_ids, field_id_array.flatten())

    tile_date_times = tile_df[tile_df['satellite_platform']=='s1']['datetime'].unique()

    X_tile = np.empty((256 * 256, 0))
    for date_time in tile_date_times[ : 4 * n_obs : n_obs]:
        vv_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VV')]['file_path'].values[0])
        vv_array = np.expand_dims(vv_src.read(1).flatten(), axis=1)

        vh_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VH')]['file_path'].values[0])
        vh_array = np.expand_dims(vh_src.read(1).flatten(), axis=1)

        X_tile = np.append(X_tile, vv_array, axis = 1)
        X_tile = np.append(X_tile, vh_array, axis = 1)

    X = np.append(X, X_tile, axis=0)

In [ ]:
data = pd.DataFrame(X)
data['label'] = y.astype(int)
data['field_id'] = field_ids
data = data[data.label != 0] #this filters the pixels that don't have a label (or corresponding field ID)
data

## Building the Model

In [50]:
# Each field has several pixels in the data. Here our goal is to build a Random Forest (RF) model using the average values
# of the pixels within each field. So, we use `groupby` to take the mean for each field_id
data_grouped = data.groupby('field_id').mean().reset_index()
data_grouped

,field_id,0,1,2,3,4,5,6,7,label
0,29.0,13.049180,3.032787,11.311475,3.344262,13.459016,3.868852,16.639344,4.721311,4.0
1,78.0,34.942857,9.085714,21.957143,5.614286,19.000000,4.614286,14.042857,3.485714,4.0
2,92.0,11.702970,4.504950,14.118812,4.207921,23.475248,6.207921,18.851485,5.415842,1.0
3,104.0,15.441284,3.189908,18.779817,4.483486,17.837615,3.785321,28.519266,8.812844,4.0
4,114.0,8.675676,2.243243,15.378378,3.081081,11.540541,3.135135,16.432432,3.783784,4.0
...,...,...,...,...,...,...,...,...,...,...
3349,122419.0,16.659292,5.216814,17.300885,4.243363,22.265487,5.699115,21.415929,5.115044,4.0
3350,122436.0,9.101974,1.532895,8.582237,2.003289,8.355263,1.723684,8.625000,1.986842,5.0
3351,122615.0,11.488889,2.385185,14.362963,2.088889,19.607407,3.511111,11.851852,3.800000,2.0
3352,122704.0,8.971173,2.785288,8.131213,2.137177,14.989066,3.524851,11.246521,3.647117,5.0


In [51]:
# Split train and test
# We use field_ids to split the data to train and test. Note that the test portion for training is different than the test 
# portion provided as part of the competition. 
train_per = 0.7

n_fields = len(data_grouped['field_id'])
np.random.seed(10)
train_fields = np.random.choice(data_grouped['field_id'], int(n_fields * train_per), replace=False)
test_fields = data_grouped['field_id'][~np.in1d(data_grouped['field_id'], train_fields)]

In [52]:
X_train, X_test = data_grouped[data_grouped['field_id'].isin(train_fields)], data_grouped[data_grouped['field_id'].isin(test_fields)]
X_train = X_train.drop(columns=['label', 'field_id'])
X_test = X_test.drop(columns=['label', 'field_id'])
y_train, y_test = data_grouped[data_grouped['field_id'].isin(train_fields)]['label'], data_grouped[data_grouped['field_id'].isin(test_fields)]['label']

In [53]:
# We ran a simple hyperparameter tuning for the number of trees, and concluded to use:
n_trees = 50

In [54]:
# Fitting the RF model
rf = RandomForestClassifier(n_estimators = n_trees, random_state = 0, n_jobs = 3)
rf.fit(X_train, y_train.astype(int))

RandomForestClassifier(n_estimators=50, n_jobs=3, random_state=0)

## Competition Test Data

In this part we will load the competition test data (which does not have labels) and predict the crop class for each field

In [55]:
tile_ids_test = competition_test_df['tile_id'].unique()

In [58]:
X_competition_test = np.empty((0, 2 * (n_obs-1)))
field_ids_test = np.empty((0, 1))

for tile_id in tile_ids_test:
    tile_df = competition_test_df[competition_test_df['tile_id']==tile_id]
    
    field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
    field_id_array = field_id_src.read(1)
    field_ids_test = np.append(field_ids_test, field_id_array.flatten())
    
    tile_date_times = tile_df[tile_df['satellite_platform']=='s1']['datetime'].unique()
    
    X_tile = np.empty((256 * 256, 0))
    for date_time in tile_date_times[ : 4 * n_obs : n_obs]:
        vv_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VV')]['file_path'].values[0])
        vv_array = np.expand_dims(vv_src.read(1).flatten(), axis=1)
        
        vh_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VH')]['file_path'].values[0])
        vh_array = np.expand_dims(vh_src.read(1).flatten(), axis=1)
        
        X_tile = np.append(X_tile, vv_array, axis = 1)
        X_tile = np.append(X_tile, vh_array, axis = 1)
        
    X_competition_test = np.append(X_competition_test, X_tile, axis=0)

In [59]:
data_test = pd.DataFrame(X_competition_test)
data_test['field_id'] = field_ids_test
data_test = data_test[data_test.field_id != 0]
data_test

,0,1,2,3,4,5,6,7,field_id
72,8.0,1.0,22.0,3.0,3.0,1.0,11.0,1.0,102896.0
73,3.0,0.0,10.0,5.0,8.0,2.0,20.0,0.0,102896.0
74,3.0,0.0,9.0,2.0,8.0,2.0,20.0,0.0,102896.0
75,6.0,0.0,12.0,1.0,12.0,1.0,12.0,0.0,102896.0
76,5.0,1.0,13.0,2.0,9.0,1.0,21.0,0.0,102896.0
...,...,...,...,...,...,...,...,...,...
6553595,7.0,0.0,3.0,1.0,23.0,1.0,31.0,6.0,88366.0
6553596,11.0,1.0,5.0,2.0,17.0,2.0,19.0,5.0,88366.0
6553597,9.0,1.0,6.0,2.0,24.0,2.0,45.0,3.0,88366.0
6553598,9.0,1.0,10.0,1.0,37.0,1.0,41.0,5.0,88366.0


In [60]:
data_test_grouped = data_test.groupby('field_id').mean().reset_index()
data_test_grouped

,field_id,0,1,2,3,4,5,6,7
0,56.0,7.124260,1.029586,11.455621,2.414201,16.183432,2.183432,17.585799,3.745562
1,60.0,28.887892,1.251121,12.334081,0.757848,13.683857,0.746637,15.367713,1.982063
2,97.0,9.765402,1.071464,7.183834,0.889601,8.770330,1.326269,9.618531,2.108428
3,103.0,6.522638,1.887795,13.256890,3.469488,9.791339,1.786417,8.295276,2.051181
4,123.0,20.945055,4.314286,24.751648,5.854945,25.162637,4.661538,34.450549,11.646154
...,...,...,...,...,...,...,...,...,...
2884,122658.0,13.574278,1.874677,24.340296,2.147853,26.417742,5.629195,12.329735,3.700070
2885,122689.0,12.496350,2.182482,11.277372,3.124088,9.744526,2.656934,14.430657,3.970803
2886,122698.0,14.734043,3.436170,13.127660,2.776596,22.159574,4.159574,9.117021,3.393617
2887,122703.0,15.899371,4.876310,17.094340,4.763103,15.155136,4.157233,21.928721,4.993711


In [61]:
y_competition_prob = rf.predict_proba(data_test_grouped.drop(columns=['field_id']))

In [62]:
# In this part we format the DataFrame to have column names and order similar to the sample submission file. 
pred_df = pd.DataFrame(y_competition_prob)
pred_df = pred_df.rename(columns={
    0:'Crop_ID_1',
    1:'Crop_ID_2', 
    2:'Crop_ID_3',
    3:'Crop_ID_4',
    4:'Crop_ID_5',
    5:'Crop_ID_6',
    6:'Crop_ID_7',
    7:'Crop_ID_8',
    8:'Crop_ID_9'
})
pred_df['field_id']=data_test_grouped['field_id']
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df

,field_id,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,Crop_ID_8,Crop_ID_9
0,56.0,0.06,0.12,0.04,0.34,0.02,0.04,0.30,0.02,0.06
1,60.0,0.26,0.06,0.14,0.10,0.06,0.14,0.10,0.10,0.04
2,97.0,0.10,0.32,0.06,0.06,0.12,0.10,0.04,0.00,0.20
3,103.0,0.04,0.26,0.02,0.08,0.16,0.24,0.00,0.04,0.16
4,123.0,0.04,0.04,0.00,0.82,0.02,0.00,0.02,0.06,0.00
...,...,...,...,...,...,...,...,...,...,...
2884,122658.0,0.00,0.04,0.02,0.00,0.00,0.14,0.74,0.06,0.00
2885,122689.0,0.04,0.20,0.02,0.48,0.10,0.02,0.06,0.06,0.02
2886,122698.0,0.04,0.28,0.04,0.24,0.10,0.08,0.10,0.00,0.12
2887,122703.0,0.06,0.00,0.02,0.88,0.00,0.02,0.00,0.00,0.02


In [63]:
# Write the predicted probabilites to a csv for submission
pred_df.to_csv('baseline_submission.csv', index=False)